In [ ]:
!pip install xlrd

In [ ]:
import pandas as pd

df = pd.read_excel('GSAF5.xls')
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df.sample(10)

In [ ]:
df.columns.to_list()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df['Fatal Y/N'].value_counts()

----------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------

RODRIGO CODE

----------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------

Cleaning 'Fatal' Data Column:

In [ ]:
df['Fatal Y/N'] = df['Fatal Y/N'].astype(str).str.strip().str.upper()

df['Fatal Y/N'] = df['Fatal Y/N'].replace({
    'N ': 'N',
    ' N': 'N',
    'UNKNOWN': 'Unknown',  
    'F': 'Unknown',
    'Y ': 'Y',
    'M': 'Unknown', 
    '2017': 'Unknown',
    '2018': 'Unknown',
    '0': 'Unknown'
})

df.loc[~df['Fatal Y/N'].isin(['Y', 'N']), 'Fatal Y/N'] = 'Unknown'



In [102]:
df['Fatal Y/N'].value_counts()

Fatal Y/N
N          4924
Y          1483
Unknown     643
Name: count, dtype: int64

Adding clean 'Injury_Class' column to the data set:

In [ ]:
df['Injury'] = df['Injury'].fillna('Unknown').astype(str).str.lower().str.strip()

def classify_injury(text):
    text = str(text).lower().strip()
    if 'no injury' in text or 'uninjured' in text:
        return 'No Injury'
    elif 'fatal' in text or 'died' in text or 'death' in text:
        return 'Fatal Wounds'
    elif any(word in text for word in [
        'minor', 'scratch', 'abrasion', 'bruise', 'small', 'superficial',
        'cut', 'scrape', 'nipped', 'grazed', 'bitten but not serious'
    ]):
        return 'Minor Wounds'
    elif any(word in text for word in [
        'amputation', 'severe', 'major', 'deep', 'critical',
        'multiple', 'serious', 'massive', 'wound', 'bites', 'bitten',
        'injury to', 'injuries to', 'damage', 'laceration', 'torn'
    ]):
        return 'Severe Wounds'
    else:
        return 'Unknown'

df['Injury_Class'] = df['Injury'].apply(classify_injury)
df['Injury_Class'] = df['Injury_Class'].str.title()

In [ ]:
df['Injury_Class'].value_counts()

In [ ]:
df['Injury_Class'].head(10)

In [ ]:
df['Injury'].value_counts()

----------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------